# このノートブックについて
- langchainで画像からデータを取り出すアプリ作成のための練習ノートブック
- 参考
  - [langchain公式チュートリアル - 基本編](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/)
  - [langchain公式チュートリアル - PDF読み込み編](https://python.langchain.com/v0.2/docs/tutorials/pdf_qa/)
  - [langchain公式チュートリアル - JSON出力](https://python.langchain.com/v0.2/docs/how_to/output_parser_json/)
- LangChainのバージョンはv0.2であることに注意 

# ライブラリインポート

In [65]:
import pandas as pd

# OpenAIのAPIキー読み込み
- APIキーを環境変数として格納する方法[Link](https://happy-shibusawake.com/api-key/1406/)

In [66]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass() # Win11のvenv環境では不要だった

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

## モデルを試しに呼び出してみる

In [67]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Japanese"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='こんにちは！', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 20, 'total_tokens': 22}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3e7d703517', 'finish_reason': 'stop', 'logprobs': None}, id='run-127e0f22-2b40-4602-a137-c40d872771ea-0', usage_metadata={'input_tokens': 20, 'output_tokens': 2, 'total_tokens': 22})

# 出力形式の指定
- StrOutputParserではstr形式で出力する。
- ほかにも、JSONなどを指定できる。

In [68]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
result = model.invoke(messages)

In [69]:
parser.invoke(result) # strを取り出し

'こんにちは！'

# Chainにする
- "|"でつなげるとChainを作成できる。
- modelの指定と、出力の指定をチェーンにする例が下記である。

In [70]:
chain = model | parser # チェーン作成
chain.invoke(messages)

'こんにちは！'

# プロンプトテンプレート
- 今回はユーザー入力のテキストを指定した言語に変換するテンプレートを作成する。

In [71]:
from langchain_core.prompts import ChatPromptTemplate

## システムメッセージテンプレート

In [72]:
system_template = "Translate the following into {language}:"

## プロンプトテンプレート
- 辞書型での定義を作成

In [73]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

## テンプレートの利用

In [74]:
result = prompt_template.invoke({"language": "Japanese", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Japanese:'), HumanMessage(content='hi')])

## パイプライン(Chain)化

In [75]:
chain = prompt_template | model | parser

In [76]:
chain.invoke({"language": "Japanese", "text": "hi"})

'こんにちは'

# PDFの読み込み
- [ここ](chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://nirouchlor.com/wp-content/uploads/2023/08/samson.pdf)に落ちていたデータシートを利用

In [77]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example_data/samson.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs)) # ページ数を出力

1


## page_contentには1文字ずつデータが入っている。

In [78]:
print(docs[0].page_content[0:1])

I


In [79]:
print(docs[0].page_content[0:100])

INSTRUMENT PROJECT: 780 NEI
DATA SHEET SPC. No. 780-FB-3090
DATE JUNE2023 Sh. 31 of  31
PLANT Chlori


### あるページのすべての文字データを取り出す

In [80]:
print(docs[0].page_content)

INSTRUMENT PROJECT: 780 NEI
DATA SHEET SPC. No. 780-FB-3090
DATE JUNE2023 Sh. 31 of  31
PLANT Chlorine Caustic Soda Plant
DESCRIPTION Pneumatic control valve with positioner Tag. LV-050850 2
SERVICE pure brine to tank D517 Q.ty 1LOCATION Line - 100-CW89-12139 P&ID N° 5-AM-1008.1/730
OPERATING CONDITION Units Min. Op. Max. Des.
FLUID PURE BRINEFLOW m3/h 75 100UP STREAM PRESSURE bar g 3 3DOWN STREAM PRESSURE bar g 1 1DIFFERENTIAL PRESSURE DROP bar 2 2SHUT-OFF PRESSURE bar g 8 8TEMPERATURE °C 85 85DENSITY K
g/m31177 1177
VISCOSITY cPs  0.89 0.89CRITICAL PRESSURE bar a - -VAPOUR PRESSURE bar a - -REQUIRED CV ACTUAL CV VALVETYPE  Plug v-port or globe BODY SIZE std. mfr.TRIM SIZE std. mfr.CONNECTION TYPE / RATING   Flanged ,DIN  PN 10BODY MATERIAL  carbon steel PTFE linedBONNET std. mfr.PACKING BOX PTFEPLUG TYPE Std. Mfr.TRIM MATERIAL std. mfr.SEAT/DESK  MATERIAL std. mfr.FLOW CHARACTERISTIC Equal %LEAKAGE (ANSI B16.104) VIFLOW DIRECTION OpenACTUATORTYPE std. mfr. SUPPLY FAILURE Valve openHA

In [81]:
print(type(docs[0].page_content))

<class 'str'>


## メタデータの取り出し

In [82]:
print(docs[0].metadata)

{'source': './example_data/samson.pdf', 'page': 0}


# PDFをJSONに変換してみる

In [85]:
system_template = (

    """You are a plant engineer.
    I will send you data extracted as text from a PDF datasheet.
    Please convert this data into the JSON format and output it.
    Ensure the output is only in the form of a data file.
    """
)

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{pdf_output}")]
)

from langchain_core.output_parsers import JsonOutputParser
parser = JsonOutputParser()

model = ChatOpenAI(model="gpt-4o")

chain = prompt_template | model | parser

output = chain.invoke({"pdf_output": docs[0].page_content})
output

{'instrument_project': '780 NEI',
 'data_sheet_spc_no': '780-FB-3090',
 'date': 'JUNE2023',
 'sheet_number': '31 of 31',
 'plant': 'Chlorine Caustic Soda Plant',
 'description': 'Pneumatic control valve with positioner',
 'tag': 'LV-050850 2',
 'service': 'pure brine to tank D517',
 'quantity': 1,
 'location': 'Line - 100-CW89-12139',
 'p_id_no': '5-AM-1008.1/730',
 'operating_condition': {'units': {'min_op': 'Min. Op.',
   'max_op': 'Max. Des.'},
  'fluid': 'PURE BRINE',
  'flow': {'min': 75, 'max': 100, 'unit': 'm3/h'},
  'up_stream_pressure': {'min': 3, 'max': 3, 'unit': 'bar g'},
  'down_stream_pressure': {'min': 1, 'max': 1, 'unit': 'bar g'},
  'differential_pressure_drop': {'min': 2, 'max': 2, 'unit': 'bar'},
  'shut_off_pressure': {'min': 8, 'max': 8, 'unit': 'bar g'},
  'temperature': {'min': 85, 'max': 85, 'unit': '°C'},
  'density': {'min': 1177, 'max': 1177, 'unit': 'kg/m3'},
  'viscosity': {'min': 0.89, 'max': 0.89, 'unit': 'cPs'},
  'critical_pressure': {'min': None, 'max'

In [87]:
import json

# ファイルに書き込み
with open('output.json', 'w') as file:
    json.dump(output, file, indent=4)
